### 🧭 Week 2: Preprocessing and Feature Engineering

**Goals**:
• Handle nulls in delay and cancellation columns

• Create derived features: Month, Day of Week, Hour, Route

• Format datetime columns

• Save preprocessed data for fast reuse


**Deliverables**:

• Cleaned dataset

• Summary of preprocessing logic

• Feature dictionary

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_PATH = Path("dataset/DelayedFlights.csv")
OUT_DIR = Path("processed")
OUT_DIR.mkdir(exist_ok=True)


In [4]:
# quick peek
df_head = pd.read_csv(DATA_PATH, nrows=5)
print(df_head.columns.tolist())
print(df_head.dtypes)


['Unnamed: 0', 'Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
Unnamed: 0             int64
Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance               int64
TaxiIn            

In [5]:
df = pd.read_csv(DATA_PATH, low_memory=False)

In [6]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [7]:
# 4a. Identify common delay columns from Week 1
delay_cols = [
    'carrierdelay', 'weatherdelay', 'nasdelay',
    'securitydelay', 'LateAircraftDelay'
]

# If names differ, adapt to actual column names in your CSV (use df.columns)
existing_delay_cols = [c for c in delay_cols if c in df.columns]

# Fill delay reason NaNs with 0
df[existing_delay_cols] = df[existing_delay_cols].fillna(0)

# 4b. Cancellation column
if 'cancellation_code' in df.columns:
    df['cancellation_code'] = df['cancellation_code'].fillna('NONE')
else:
    # sometimes column is 'canc_code' or similar - keep safe
    pass

# 4c. Small-missing numeric fields: use median
small_missing_num = ['actualelapsedtime', 'airtime', 'arrdelay', 'deptime', 'arrtime', 'taxiin', 'taxiout']
small_missing_num = [c for c in small_missing_num if c in df.columns]
for c in small_missing_num:
    if df[c].isna().mean() > 0 and df[c].isna().mean() <= 0.05:
        df[c] = df[c].fillna(df[c].median())


In [8]:
df['any_delay_reason'] = (df[existing_delay_cols].sum(axis=1) > 0).astype('int8')


In [9]:
def hhmm_to_time_str(x):
    if pd.isna(x): return np.nan
    # Some values might be floats (e.g., 930.0)
    try:
        x = int(x)
    except:
        return np.nan
    h = x // 100
    m = x % 100
    if h >= 24 or m >= 60:  # handle malformed
        return np.nan
    return f"{h:02d}:{m:02d}"


In [10]:
# Create fl_date from Year, Month, DayofMonth
if 'year' in df.columns and 'month' in df.columns and 'dayofmonth' in df.columns:
    df['fl_date'] = pd.to_datetime(
        df[['year', 'month', 'dayofmonth']].rename(columns={
            'year': 'year', 'month': 'month', 'dayofmonth': 'day'
        }), 
        errors='coerce'
    )
elif 'fl_date' in df.columns:
    df['fl_date'] = pd.to_datetime(df['fl_date'], errors='coerce')
else:
    print("Warning: Could not find date columns")

# convert dep/arr times
for tcol, outcol in [('deptime', 'dep_time_str'), ('arrtime', 'arr_time_str')]:
    if tcol in df.columns:
        df[outcol] = df[tcol].apply(hhmm_to_time_str)

# combine
if 'dep_time_str' in df.columns and 'fl_date' in df.columns:
    df['dep_datetime'] = pd.to_datetime(
        df['fl_date'].dt.strftime('%Y-%m-%d') + ' ' + df['dep_time_str'],
        errors='coerce'
    )
if 'arr_time_str' in df.columns and 'fl_date' in df.columns:
    df['arr_datetime'] = pd.to_datetime(
        df['fl_date'].dt.strftime('%Y-%m-%d') + ' ' + df['arr_time_str'],
        errors='coerce'
    )

# If arrival passes midnight (arr < dep), you may need logic to add one day:
if 'dep_datetime' in df.columns and 'arr_datetime' in df.columns:
    mask_nextday = (df['dep_datetime'].notna() & df['arr_datetime'].notna() &
                    (df['arr_datetime'] < df['dep_datetime']))
    df.loc[mask_nextday, 'arr_datetime'] += pd.Timedelta(days=1)


In [11]:
# MONTH, DAY_OF_WEEK
df['month'] = df['fl_date'].dt.month.astype('Int8')
df['day_of_week'] = df['fl_date'].dt.day_name()

# HOUR (use scheduled departure hour if present)
if 'dep_datetime' in df.columns:
    df['dep_hour'] = df['dep_datetime'].dt.hour.astype('Int8')
elif 'crsdeptime' in df.columns:
    # crsdeptime = scheduled time
    df['dep_hour'] = df['crsdeptime'].apply(hhmm_to_time_str).pipe(pd.to_datetime, errors='coerce').dt.hour

# ROUTE
if 'origin' in df.columns and 'dest' in df.columns:
    df['route'] = df['origin'].astype(str) + '-' + df['dest'].astype(str)

# IS_DELAYED flag: common threshold = arrival delay > 15 minutes
if 'arrdelay' in df.columns:
    df['is_delayed'] = (df['arrdelay'] > 15).astype('int8')
else:
    # fallback using any_delay_reason
    df['is_delayed'] = df['any_delay_reason'].astype('int8')


In [12]:
# Extra useful features:

# Delay magnitude (total delay)
df['total_delay_minutes'] = df[existing_delay_cols].sum(axis=1)

# departure delay flag
if 'depdelay' in df.columns:
    df['dep_delayed'] = (df['depdelay'] > 15).astype('int8')

# flight duration from datetimes if available
if 'dep_datetime' in df.columns and 'arr_datetime' in df.columns:
    df['scheduled_duration_min'] = (df['arr_datetime'] - df['dep_datetime']).dt.total_seconds() / 60
    # if elapsed time exists use actualelapsedtime
    if 'actualelapsedtime' in df.columns:
        df['actual_duration_min'] = df['actualelapsedtime']


In [13]:
# Memory usage before optimization
print("Memory BEFORE optimization (MB):", df.memory_usage(deep=True).sum() / 1024**2)


Memory BEFORE optimization (MB): 1475.8556070327759


In [14]:
# convert low-cardinality strings to category
cat_columns = ['uniquecarrier', 'origin', 'dest', 'cancellation_code', 'route', 'day_of_week']
cat_columns = [c for c in cat_columns if c in df.columns]
for c in cat_columns:
    df[c] = df[c].astype('category')

# downcast numeric columns
for c in df.select_dtypes(include=['int64']).columns:
    df[c] = pd.to_numeric(df[c], downcast='signed')
for c in df.select_dtypes(include=['float64']).columns:
    df[c] = pd.to_numeric(df[c], downcast='float')


In [15]:
# Memory usage after optimization
print("Memory AFTER optimization (MB):", df.memory_usage(deep=True).sum() / 1024**2)


Memory AFTER optimization (MB): 683.8420162200928


### 8️⃣ Validation Checks (Sanity Tests)

In [16]:
# No NaNs in delay reason columns
for c in existing_delay_cols:
    assert df[c].isna().sum() == 0, f"{c} still has NAs"

print("✅ All delay reason columns have no NaNs")

# Cancellation code filled
if 'cancellation_code' in df.columns:
    print("\nCancellation unique values:")
    print(df['cancellation_code'].value_counts().head())

# Basic distributions
print("\n📊 Key Metrics:")
print(f"Delay > 15 min %: {df['is_delayed'].mean() * 100:.2f}%")
print(f"Any delay reason present %: {df['any_delay_reason'].mean() * 100:.2f}%")


✅ All delay reason columns have no NaNs

📊 Key Metrics:
Delay > 15 min %: 63.17%
Any delay reason present %: 54.78%


### 9️⃣ Save Processed Dataset for Fast Reuse

In [ ]:
# Single parquet file (fast and compressed)
df.to_parquet(OUT_DIR / "delayedflights_2008_processed.parquet", index=False)
print("✅ Saved main processed file: delayedflights_2008_processed.parquet")

# Save partitioned by month (good for later chunked read)
# df.to_parquet(OUT_DIR / "delayedflights_2008_by_month.parquet", index=False, partition_cols=['month'])
# print("✅ Saved partitioned file: delayedflights_2008_by_month.parquet")


In [ ]:
# Save a small sample for quick prototyping
df.sample(10000, random_state=42).to_parquet(OUT_DIR / "sample_10k.parquet", index=False)
print("✅ Saved sample file: sample_10k.parquet (10,000 rows)")


✅ Saved sample file: sample_10k.parquet (10,000 rows)


In [ ]:
# Display file information
import os

print("\n📁 Saved Files Summary:")
print("-" * 50)
for file in OUT_DIR.glob("*.parquet"):
    file_size = os.path.getsize(file) / 1024**2  # MB
    print(f"{file.name}: {file_size:.2f} MB")



📁 Saved Files Summary:
--------------------------------------------------
delayedflights_2008_processed.parquet: 83.92 MB
sample_10k.parquet: 0.70 MB


In [ ]:
# Create comprehensive feature dictionary
feature_dict = {
    'Feature Name': [
        'fl_date',
        'dep_datetime',
        'arr_datetime',
        'month',
        'day_of_week',
        'dep_hour',
        'origin',
        'dest',
        'route',
        'carrierdelay',
        'weatherdelay',
        'nasdelay',
        'securitydelay',
        'lateaircraftdelay',
        'total_delay_minutes',
        'any_delay_reason',
        'is_delayed',
        'dep_delayed',
        'cancellation_code',
        'scheduled_duration_min',
        'actual_duration_min'
    ],
    'Data Type': [
        'datetime',
        'datetime',
        'datetime',
        'int (1–12)',
        'category',
        'int (0–23)',
        'category',
        'category',
        'category',
        'float',
        'float',
        'float',
        'float',
        'float',
        'float',
        'int8 (0/1)',
        'int8 (0/1)',
        'int8 (0/1)',
        'category',
        'float',
        'float'
    ],
    'Description': [
        'Flight date (YYYY-MM-DD)',
        'Combined departure datetime (fl_date + deptime)',
        'Combined arrival datetime (fl_date + arrtime, +1 day if crossing midnight)',
        'Month of flight',
        'Day name (Monday, Tuesday, etc.)',
        'Departure hour',
        'Origin airport code (e.g., JFK)',
        'Destination airport code (e.g., LAX)',
        'Origin-Destination pair (e.g., JFK-LAX)',
        'Minutes of carrier delay (0 if none)',
        'Minutes of weather delay (0 if none)',
        'Minutes of NAS delay (0 if none)',
        'Minutes of security delay (0 if none)',
        'Minutes of late aircraft delay (0 if none)',
        'Sum of all reason delays',
        'Any delay reason present flag',
        'Arrival delay > 15 minutes flag',
        'Departure delay > 15 minutes flag',
        "'NONE' when not cancelled; other codes for cancellation reasons",
        'Duration computed from datetimes (minutes)',
        'Actual elapsed time if present (minutes)'
    ]
}

feature_df = pd.DataFrame(feature_dict)
print("📖 FEATURE DICTIONARY")
print("=" * 120)
print(feature_df.to_string(index=False))
print("\n" + "=" * 120)
print(f"Total Features Documented: {len(feature_df)}")


📖 FEATURE DICTIONARY
          Feature Name  Data Type                                                                Description
               fl_date   datetime                                                   Flight date (YYYY-MM-DD)
          dep_datetime   datetime                            Combined departure datetime (fl_date + deptime)
          arr_datetime   datetime Combined arrival datetime (fl_date + arrtime, +1 day if crossing midnight)
                 month int (1–12)                                                            Month of flight
           day_of_week   category                                           Day name (Monday, Tuesday, etc.)
              dep_hour int (0–23)                                                             Departure hour
                origin   category                                            Origin airport code (e.g., JFK)
                  dest   category                                       Destination airport code (e.g., LAX

In [ ]:
# Final summary statistics
print("📈 PREPROCESSING SUMMARY")
print("=" * 80)
print(f"📁 Dataset: DelayedFlights (2008)")
print(f"📊 Total Records: {len(df):,}")
print(f"📋 Total Columns: {len(df.columns)}")
print(f"💾 Final Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"🔢 Features Engineered: {len(['fl_date', 'dep_datetime', 'arr_datetime', 'month', 'day_of_week', 'dep_hour', 'route', 'total_delay_minutes', 'any_delay_reason', 'is_delayed', 'dep_delayed', 'scheduled_duration_min', 'actual_duration_min'])}")
print(f"\n✅ Missing Values Handled: {len(existing_delay_cols)} delay columns + cancellation codes")
print(f"✅ Datetime Features: 3 (fl_date, dep_datetime, arr_datetime)")
print(f"✅ Temporal Features: 3 (month, day_of_week, dep_hour)")
print(f"✅ Categorical Features: 4 (origin, dest, route, day_of_week)")
print(f"✅ Delay Indicators: 4 (is_delayed, any_delay_reason, dep_delayed, total_delay_minutes)")
print(f"✅ Duration Features: 2 (scheduled_duration_min, actual_duration_min)")
print("=" * 80)

📈 PREPROCESSING SUMMARY
📁 Dataset: DelayedFlights (2008)
📊 Total Records: 1,936,758
📋 Total Columns: 44
💾 Final Memory Usage: 683.84 MB
🔢 Features Engineered: 13

✅ Missing Values Handled: 4 delay columns + cancellation codes
✅ Datetime Features: 3 (fl_date, dep_datetime, arr_datetime)
✅ Temporal Features: 3 (month, day_of_week, dep_hour)
✅ Categorical Features: 4 (origin, dest, route, day_of_week)
✅ Delay Indicators: 4 (is_delayed, any_delay_reason, dep_delayed, total_delay_minutes)
✅ Duration Features: 2 (scheduled_duration_min, actual_duration_min)
💾 Final Memory Usage: 683.84 MB
🔢 Features Engineered: 13

✅ Missing Values Handled: 4 delay columns + cancellation codes
✅ Datetime Features: 3 (fl_date, dep_datetime, arr_datetime)
✅ Temporal Features: 3 (month, day_of_week, dep_hour)
✅ Categorical Features: 4 (origin, dest, route, day_of_week)
✅ Delay Indicators: 4 (is_delayed, any_delay_reason, dep_delayed, total_delay_minutes)
✅ Duration Features: 2 (scheduled_duration_min, actual_du